In [ ]:
import numpy as np
import pandas as pd


Problem11. ``'Military Expenditure.csv'`` ֆայլի տվյալները կարդացեք DataFrame-ով և վերադարձրեք առաջին 10 տողերը, ինչպես նաև ամբողջ DataFrame-ը։

In [ ]:
def read_return_top10(file_name):
    data = pd.read_csv(file_name)
    df = pd.DataFrame(data)
    dfFirst = df[:10]
    return dfFirst, df


In [ ]:
# Do not modify this cell
top10, df = read_return_top10('Military Expenditure.csv')
assert top10.iloc[8]['Name'] == 'Armenia'
assert top10.shape == (10, 63)

Military Expenditure dataset-ը պատրաստվել է Ստոկհոլմի խաղաղության միջազգային հետազոտությունների ինստիտուտի (SIPRI) կողմից և պարունակում է ավելի քան 200 պետությունների 1960-2018 թվականներին կատարած ռազմական ծախսերը դոլարներով։

Problem12. Ջնջել այն տողերը որոնց 'Type' սյունը "Country" *չէ*։

In [ ]:
def remove_non_countries(df):
    index = df[df['Type']!='Country'].index
    df.drop(index,inplace=True)
    


In [ ]:
assert remove_non_countries(df) is None
assert df.Type.nunique() == 1
assert df.shape == (202, 63)

Problem13. Ջնջել 'Code', 'Type', 'Indicator Name' սյուները։

In [ ]:
def remove_some_cols(df):
  df.drop(['Code', 'Type','Indicator Name'], axis=1, inplace=True)

In [ ]:
# Do not modify this cell
assert remove_some_cols(df) is None
assert df.shape == (202, 60)
assert df.columns[1] == '1960'

Problem14. Ստացված dataframe-ը գրել(պահպանել) 'MilitaryExpenditureSmall.csv' ֆայլի վրա **այնպես, ինչպես որ կա**։

In [ ]:
def save_df(df, file_name):
  df.to_csv(file_name,index=False)

In [ ]:
# Do not modify this cell
file_name = 'MilitaryExpenditureSmall.csv'
save_df(df, file_name)
small_df = pd.read_csv(file_name)

assert small_df.iloc[0, 0] == 'Aruba'
assert small_df.shape[1] == 60

Problem15. Վերադարձնել Հայաստանին համապատասխանող տողի այն տվյալները/վանդակները որոնք դատարկ չեն։

In [ ]:
def info_arm(df):
  index = df[df['Name'] != 'Armenia'].index
  test = df.drop(index)
  test.dropna(axis = 1,inplace = True  )
  return test

In [ ]:
arm = info_arm(df)
assert arm.shape == (1, 26)
assert arm.iloc[:, 1:].max(axis=1).values[0] == 608854649.9
assert '1994' not in arm.columns

Problem16. Վերադարձնել DataFrame, որտեղ կլինի պետության անունը՝ որպես ինդեքս և յուրաքանչյուր պետության միջին տարեկան ծախսը՝ որպես ``'mean_expense'`` սյուն՝ գրված նվազման կարգով։  Վերջնական աղյուսակում չպետք է լինեն այնպիսի երկրներ, որոնց ծախսերի տվյալները ընդհանրապես առկա չեն կամ 0 են։

In [ ]:
def per_country_mean_expense(df):
  a = pd.DataFrame(data = df.mean(axis=1), columns = ['mean_expense'])
  a.index = df['Name']
  a.dropna(inplace = True)
  a = a.loc[a['mean_expense'] != 0]
  a = a.sort_values('mean_expense',ascending= False)
  return a

In [ ]:
# Do not modify this cell
grouped_df = per_country_mean_expense(df)
assert grouped_df.loc['Armenia'][0] == arm.iloc[:,1:].mean(axis=1).values[0]
assert grouped_df.shape == (164, 1)
assert grouped_df.index[0] == 'United States'
assert grouped_df.index[-1] == 'Haiti'

Problem17. Տրված է ՝ $10 \times 4$ չափնի ```pandas.DataFrame (df)```։ Հարկավոր է գրել ֆունկցիա, որը կստեղծի նոր ```pandas.DataFrame```, որի մեջ կլինեն ```df```-ի տողերն ու սյուները և 2 նոր սյուն ```nearest_row```  և ```dist``` որոնց մեջ գրված կլինեն ամենամոտիկ տողի ```index```-ը և էվկլիդյան հեռավորությունը այդ տողերի միջև։ 


In [ ]:
def add_columns(df):
  temp = df.copy()
  nearest_rows = []
  nearest_distance = []
  for i, row in temp.iterrows():
        curr = row
        rest = temp.drop(i)
        e_dists = {}
        for j, x in rest.iterrows():
            e_dists.update({j: round(np.linalg.norm(curr.values - x.values))})
        nearest_rows.append(min(e_dists, key=e_dists.get))
        nearest_distance.append(min(e_dists.values()))
  temp['nearest_row'] = nearest_rows
  temp['dist'] = nearest_distance
  return temp

In [ ]:
np.random.seed(42)
df = pd.DataFrame(np.random.randint(1, 100, 40).reshape(10, -1),
                  columns=list('pqrs'),
                  index=list('abcdefghij'))
new_df = add_columns(df)
assert isinstance(new_df, pd.DataFrame)
assert new_df.loc['a']['nearest_row'] == 'g'
assert new_df.nearest_row.value_counts()[0] == 4
assert id(new_df) != id(df)

Problem18. Տրված է՝ $93 \times 27 $ չափանի ```pandas.DataFrame (df)```։ Հարկավոր է գրել ֆունկցիա, որը կստեղծի նոր ```pandas.DataFrame```, որի մեջ կլինեն ```df```-ի տողերն ու սյուները, բացառությամբ ```Type``` սյան, որի փոխարեն կլինեն մի քանի սյուներ՝ ստացված HW4-ի 4-րդ խնդրի ֆունկցիայի միջոցով։ Սյուների անունները կլինեն ```Type``` սյան արժեքները (Midsize, Small, ...):


In [ ]:
def convert(df):
  df = pd.DataFrame(df)
  coloumn = np.sort(df['Type'].dropna().unique())
  df = df.drop(['Type'], axis = 1)
  df[coloumn] = np.resize(df, (93,6))
  return df

In [ ]:
# Do not modify this cell
df = pd.read_csv(
    'https://raw.githubusercontent.com/selva86/datasets/master/Cars93_miss.csv')
new_df = convert(df)
# assert convert(df).iloc[0]['Small'] == 1
assert isinstance(new_df, pd.DataFrame)
assert id(new_df) != id(df)
assert new_df.shape == (93, 32)
assert new_df.columns[-5:].tolist() == ['Large', 'Midsize',
                                        'Small', 'Sporty', 'Van']

Problem19. Նախորդ խնդրում ստացված dataset-ի  ```Min.price``` և ```Max.price``` սյուների բացակայող արժեքները լրացնել համապատասխանաբար այդ սյուների  միջինով և մեդիանով՝ օգտագործելով ``apply()`` մեթոդը:


In [ ]:
def fill_NaN(df):
  df = df.fillna(df[["Min.Price"]].mean())
  df = df.fillna(df[["Max.Price"]].median())
  return df

In [ ]:
# Do not modify this cell
assert isinstance(new_df, pd.DataFrame)
other_df = fill_NaN(new_df)
assert abs(other_df.iloc[3]['Min.Price'] - 17.118605) < 0.0001
assert abs(other_df.iloc[4]['Max.Price'] - 19.15) < 0.0001